**POZNÁMKA: Tento notebook je určený pre platformu Google Colab, ktorá zdarma poskytuje hardvérovú akceleráciu. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook, pomocou lokálnej grafickej karty.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install torchinfo
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from class_utils.pytorch_utils import BestModelCheckpointer, freeze_except_last
from torch.optim.lr_scheduler import ExponentialLR
from torchvision import models, transforms
from torch.utils.data import DataLoader, TensorDataset
from torchvision.datasets import ImageFolder
import torchinfo
import torch.nn as nn
import torch

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract
download_file_maybe_extract("https://www.dropbox.com/s/w4pg809npvatye0/food5v2.zip?dl=1", directory="data/food5v2")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Transfer učenie: Predtrénovaná sieť ako extraktor príznakov

V predchádzajúcom notebooku sme sa venovali štandardnému typu transfer učenia so zmrazenými váhami, trénovaním novej záverečnej vrstvy a následným dolaďovaním váh. Existuje však aj iná alternatíva, kde sa predtrénovaná sieť používa ako extraktor príznakov. Pri tomto prístupe by sme najskôr odstránili klasifikačnú vrstvu siete, aby sieť vracala vektor príznakov z predposlednej vrstvy a nie logity z poslednej vrstvy. Potom by sme s ňou prešli celú dátovú množinu a predspracovali ju. Vďaka tomu bude naša dátová množina oveľa, oveľa menšia – pokiaľ nebola už na začiatku extrémne veľká, mala by sa dokonca zmestiť do pamäte celá naraz.

Následne môžeme použiť predspracované dáta a trénovať len nové vrstvy siete – trénovanie prebehne oveľa rýchlejšie, pretože nebude potrebné znova a znova načítavať obrázky a aplikovať na ne celú veľkú sieť. Jednou nevýhodou je, samozrejme, to, že nie je možné využiť techniky zväčšovania dátovej množiny (data augmentation) – to však nemusí byť príliš vysoká cena za omnoho rýchlejší tréning.

### Zostavenie data loader-ov

Keďže nebudeme používať zväčšovanie dátovej množiny, môžeme na transformáciu obrázkov použiť priamo transformačné funkcie pribalené k predtrénovanýn váham. Okrem toho budú dátové množiny a data loader-y konštruované rovnakým spôsobom, ako sme ich konštruovali v predchádzajúcom príklade.



In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
weights = models.ResNet50_Weights.IMAGENET1K_V2
image_transforms = weights.transforms()

In [ ]:
train_dataset = ImageFolder(
    "data/food5v2/training",
    image_transforms
)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4
)

valid_dataset = ImageFolder(
    "data/food5v2/validation",
    image_transforms
)

valid_dataloader = DataLoader(
    valid_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4
)

test_dataset = ImageFolder(
    "data/food5v2/testing",
    image_transforms
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4
)

### Načítavanie predtrénovanej siete

Ďalej načítame predtrénovanú sieť ResNet50. Aby sme odstránili poslednú vrstvu (`.fc`), nahradíme ju prázdnym modulom `nn.Sequential`.



In [ ]:
pretrained_net = models.resnet50(weights=weights).to(device)
pretrained_net.fc = nn.Sequential()

### Predspracovanie dát

Použiť sieť na predspracovanie dát je vcelku jednoduché. Pre každú časť (trénovaciu, validačnú, testovaciu) len iterujeme cez data loader a zhromaždíme požadované výstupy a predspracované vstupy do dvoch tenzorov. Potom zostavíme objekt `TensorDataset` a ďalší zodpovedajúci data loader pre každú časť dát.



In [ ]:
def extract_features(feature_extractor, data_loader):
    feature_extractor.eval()
    X = []; Y = []

    for X_batch, Y_batch in data_loader:
        X_batch = X_batch.to(device)

        with torch.no_grad():
            X_batch = feature_extractor(X_batch)

        X.extend(X_batch.cpu())
        Y.extend(Y_batch.cpu())
  
    return torch.stack(X), torch.stack(Y)

In [ ]:
X_train, Y_train = extract_features(pretrained_net, train_dataloader)
X_valid, Y_valid = extract_features(pretrained_net, valid_dataloader)
X_test, Y_test = extract_features(pretrained_net, test_dataloader)

train_tensor_dataset = TensorDataset(X_train, Y_train)
train_tensor_dataloader = DataLoader(train_tensor_dataset, batch_size=32, shuffle=True)
valid_tensor_dataset = TensorDataset(X_valid, Y_valid)
valid_tensor_dataloader = DataLoader(valid_tensor_dataset, batch_size=32, shuffle=True)
test_tensor_dataset = TensorDataset(X_test, Y_test)
test_tensor_dataloader = DataLoader(test_tensor_dataset, batch_size=32, shuffle=True)

### Tréning nových vrstiev

Nové zakončenie našej siete bude vyzerať rovnako ako v predchádzajúcom príklade. Tréningová slučka bude tiež celkom štandardná – až na to, že teraz budeme iterovať cez `train_tensor_dataloader` a zakončenie siete budeme trénovať samostatne, t.j. nebude pripojené k predtrénovanej sieti. Keďže tréning bude teraz oveľa rýchlejší, môžeme si dovoliť zvýšiť aj počet epoch.



In [ ]:
class ModelTop(nn.Module):
    def __init__(self, num_features, num_outputs):
        super().__init__()
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(num_features, num_outputs)

    def set_dropout(self, p):
        self.dropout.p = p
    
    def forward(self, x):
        y = torch.flatten(x, 1)
        y = self.dropout(y)
        y = self.fc(y)
        return y

In [ ]:
model = ModelTop(X_train.shape[1], 10).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
checkpointer = BestModelCheckpointer(checkpoint_path="output/best_model.pt")
loss_train = []
loss_valid = []

for epoch in range(200):
    epoch_train_loss = []
    epoch_valid_loss = []

    model.train()
    for X_batch, Y_batch in train_tensor_dataloader:
        X_batch = X_batch.to(device)
        Y_batch = Y_batch.to(device)
        
        y_batch = model(X_batch)
        loss = criterion(y_batch, Y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_train_loss.append(loss.item())

    loss_train.append(np.mean(epoch_train_loss))

    model.eval()
    for X_batch, Y_batch in valid_tensor_dataloader:
        X_batch = X_batch.to(device)
        Y_batch = Y_batch.to(device)
        
        with torch.no_grad():
            y_batch = model(X_batch)
            loss = criterion(y_batch, Y_batch)

        epoch_valid_loss.append(loss.item())

    loss_valid.append(np.mean(epoch_valid_loss))
    checkpointer(loss_valid[-1], model)

    if epoch % 5 == 0:
        print(f"epoch {epoch}, train loss: {np.mean(loss_train[-5:])}, valid loss: {np.mean(loss_valid[-5:])}")

print(f"epoch {epoch}, loss: {loss_train[-1]}")

In [ ]:
plt.plot(loss_train, label="train")
plt.plot(loss_valid, label="valid")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.grid(ls='--')
plt.legend()

### Evaluácia

Po dokončení tréningu môžeme načítať váhy s najlepšou validačnou chybou a vyhodnotiť model na testovacích dátach.



In [ ]:
model.load_state_dict(torch.load("output/best_model.pt"));

In [ ]:
eval_Y = []
eval_y = []

model.eval()
for X_batch, Y_batch in test_tensor_dataloader:
    eval_Y.extend(Y_batch.numpy())
    X_batch = X_batch.to(device)
    Y_batch = Y_batch.to(device)
    
    with torch.no_grad():
        y_batch = model(X_batch)

    eval_y.extend(y_batch.argmax(dim=1).cpu().numpy())

eval_Y = np.array(eval_Y)
eval_y = np.array(eval_y)

cm = pd.crosstab(
    eval_Y, eval_y,
    rownames=['actual'],
    colnames=['predicted']
)
print(cm, '\n')

acc = accuracy_score(eval_Y, eval_y)
print("Accuracy = {}".format(acc))

### Opätovné zloženie modelu

Nakoniec môžeme model opäť zložiť dokopy, aby sme ho vedeli spustiť aj na pôvodných dátach. Je to veľmi jednoduché – je potrebné iba priradiť náš `model` do `pretrained_net.fc`.



In [ ]:
pretrained_net.fc = model

In [ ]:
eval_Y = []
eval_y = []

pretrained_net.eval()
for X_batch, Y_batch in test_dataloader:
    eval_Y.extend(Y_batch.numpy())
    X_batch = X_batch.to(device)
    Y_batch = Y_batch.to(device)
    
    with torch.no_grad():
        y_batch = pretrained_net(X_batch)

    eval_y.extend(y_batch.argmax(dim=1).cpu().numpy())

eval_Y = np.array(eval_Y)
eval_y = np.array(eval_y)

cm = pd.crosstab(
    eval_Y, eval_y,
    rownames=['actual'],
    colnames=['predicted']
)
print(cm, '\n')

acc = accuracy_score(eval_Y, eval_y)
print("Accuracy = {}".format(acc))